<a href="https://colab.research.google.com/github/jihun0423/Pipeline-Config/blob/main/ML_%26_DL_pipeline_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# encoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler


from sklearn.preprocessing import MinMaxScaler

# machine learning models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor

!pip install pycaret
from pycaret.regression import *

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

!pip install skorch

from skorch import NeuralNetRegressor
from sklearn.base import BaseEstimator, TransformerMixin

import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, KFold
from sklearn import set_config

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 6.7 MB/s eta 0:00:0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd/content/gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
import pandas as pd
df=pd.read_csv("london_merged.csv")

In [ ]:
df

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['month'] = df['timestamp'].dt.month
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.dayofweek

In [ ]:
features = df.drop(columns=['timestamp','cnt'])
target = df['cnt']

In [ ]:
split = int(0.8*len(features))
X_train = features[:split]
X_test = features[split:]
y_train = target[:split]
y_test = target[split:]

In [ ]:
cols = features.columns

In [ ]:
cat_cols = ['weather_code','is_holiday','is_weekend','season','month','hour','day']

In [ ]:
cols

Index(['t1', 't2', 'hum', 'wind_speed', 'weather_code', 'is_holiday',
       'is_weekend', 'season', 'month', 'hour', 'day'],
      dtype='object')

In [ ]:
def pipeline(cols,cat_cols):
  cat_transformer = OneHotEncoder(sparse=False,handle_unknown='ignore')

  num_cols = list(set(cols)-set(cat_cols))
  num_cols.sort()
  num_transformer = RobustScaler()

  preprocessor = ColumnTransformer(transformers=[("num",num_transformer,num_cols),
                                                 ("cat",cat_transformer,cat_cols)])

  model = Pipeline(steps=[("preprocessor",preprocessor)])

  return model

In [ ]:
from sklearn import set_config
set_config(display='diagram')
pp = pipeline(list(cols),list(cat_cols))

NameError: ignored

In [ ]:
pp

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['hum', 't1', 't2',
                                                   'wind_speed']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['weather_code', 'is_holiday',
                                                   'is_weekend',
                                                   'season'])]))])

In [ ]:
features_pp = pp['preprocessor'].fit_transform(features)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
features_pp = pd.DataFrame(features_pp)

In [ ]:
split = int(0.8*len(features))
X_train = features_pp[:split]
X_test = features_pp[split:]
y_train = target[:split]
y_test = target[split:]

In [ ]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 KB 14.3 MB/s eta 0:00:00


In [ ]:
from skorch import NeuralNetRegressor
from sklearn.base import BaseEstimator, TransformerMixin

def pipeline(cols, cat_cols, method='lr'):
  cat_transformer = OneHotEncoder(sparse=False,handle_unknown='ignore')

  num_cols = list(set(cols)-set(cat_cols))
  num_cols.sort()
  num_transformer = RobustScaler()

  preprocessor = ColumnTransformer(transformers=[("num",num_transformer,num_cols),
                                                 ("cat",cat_transformer,cat_cols)])


  class FloatTransformer(BaseEstimator, TransformerMixin):
      def __init__(self):
            pass
      def fit(self, X, y=None):
            return self
      def transform(self, x):
            return np.array(x, dtype=np.float32)

  if method == 'lr':
    ml = LinearRegression(fit_intercept=True)
  elif method == 'rf':
    ml = RandomForestRegressor()

  model = Pipeline(steps=[("preprocessor",preprocessor),
                          ("float64to32",FloatTransformer()),
                          ("ml",ml)])

  return model

In [ ]:
from sklearn import set_config
set_config(display='diagram')
pp = pipeline(list(cols),list(cat_cols),method='rf')

In [ ]:
pp

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['hum', 't1', 't2',
                                                   'wind_speed']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['weather_code', 'is_holiday',
                                                   'is_weekend', 'season',
                                                   'month', 'hour', 'day'])])),
                ('float64to32', FloatTransformer()),
                ('ml', RandomForestRegressor())])

In [ ]:
pp.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['hum', 't1', 't2',
                                                   'wind_speed']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['weather_code', 'is_holiday',
                                                   'is_weekend', 'season',
                                                   'month', 'hour', 'day'])])),
                ('float64to32', FloatTransformer()),
                ('ml', RandomForestRegressor())])

In [ ]:
y_pred = pp.predict(X_test)

In [ ]:
import numpy as np
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_rf

320.08306319907257

In [ ]:
set_config(display='diagram')
pp = pipeline(list(cols),list(cat_cols),method='lr')
pp.fit(X_train,y_train)
y_pred = pp.predict(X_test)
rmse_lr = np.sqrt(mean_squared_error(y_test,y_pred))
rmse_lr

626.154518714597

In [ ]:
from sklearn import set_config
set_config(display='diagram')
pp = pipeline(list(cols),list(cat_cols),method='rf')
pp

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['hum', 't1', 't2',
                                                   'wind_speed']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['weather_code', 'is_holiday',
                                                   'is_weekend', 'season',
                                                   'month', 'hour', 'day'])])),
                ('float64to32', FloatTransformer()),
                ('ml', RandomForestRegressor())])

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold


In [ ]:
params = {
    'ml__n_estimators': [10,100,200,300],
    'ml__min_samples_leaf' : [1,5,10],
    'ml__max_features' : [1,'sqrt','log2']
}

In [ ]:
grid_model=GridSearchCV(estimator=pp,param_grid=params,scoring='neg_root_mean_squared_error').fit(X_train,y_train)

In [ ]:
print(grid_model.best_params_)
grid_model.best_score_

{'ml__max_features': 'sqrt', 'ml__min_samples_leaf': 1, 'ml__n_estimators': 300}


-344.31824333780503

In [ ]:
y_pred=grid_model.predict(X_test)
rmse_gr_rf = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_gr_rf

394.02687763379043

In [ ]:
X_train

,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,month,hour,day
0,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0,1,0,6
1,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0,1,1,6
2,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0,1,2,6
3,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0,1,3,6
4,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0,1,4,6
...,...,...,...,...,...,...,...,...,...,...,...
13926,16.0,16.0,55.0,19.0,1.0,0.0,0.0,1.0,8,22,1
13927,15.0,15.0,59.0,16.0,1.0,0.0,0.0,1.0,8,23,1
13928,14.5,14.5,63.0,17.0,1.0,0.0,0.0,1.0,8,0,2
13929,14.0,14.0,67.0,14.0,1.0,0.0,0.0,1.0,8,1,2


In [ ]:
pp = pipeline(list(cols),list(cat_cols),method='rf')
X_train_pp = pp["preprocessor"].fit_transform(X_train)
print(X_train_pp.shape)
X_train_pp[0]

(13931, 62)


array([ 0.975     , -1.1875    , -1.05      , -0.90909091,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt


In [ ]:
class Regressor(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(62,30,bias=True)
    self.fc0 = nn.Linear(30,30,bias=True)
    self.fc2 = nn.Linear(30,10,bias=True)
    self.fc3 = nn.Linear(10,1,bias=True)
    self.dropout = nn.Dropout(0.2)

  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc0(x))
    x = self.dropout(F.relu(self.fc2(x)))
    x = F.relu(self.fc3(x))

    return x

In [ ]:
class Regressor(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(nn.Linear(62,30),
                               nn.ReLU(),
                               nn.Linear(30,30),
                               nn.ReLU(),
                               nn.Linear(30,10),
                               nn.Dropout(0.2),
                               nn.ReLU(),
                               nn.Linear(10,1),
                               )

  def forward(self,x):
    x=self.model(x)
    return x

In [ ]:
model = Regressor()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [ ]:
df = pd.read_pickle('df.pkl')

In [ ]:
split = int(0.8*len(df))
train = df.iloc[:split,1:]
test = df.iloc[split:,1:]


In [ ]:
setup_rgs = setup(data=train,target='cnt')

,Description,Value
0,Session id,5553
1,Target,cnt
2,Target type,Regression
3,Original data shape,"(13783, 63)"
4,Transformed data shape,"(13783, 63)"
5,Transformed train set shape,"(9648, 63)"
6,Transformed test set shape,"(4135, 63)"
7,Ordinal features,58
8,Numeric features,4
9,Categorical features,58


In [ ]:
top5 = compare_models(sort='MSE',n_select=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,149.4491,61131.4244,246.6449,0.9460,0.4444,0.3029,4.8080
et,Extra Trees Regressor,143.7931,72337.6512,268.1057,0.9363,0.2863,0.2112,9.7980
rf,Random Forest Regressor,150.5539,75346.5790,273.5923,0.9336,0.3034,0.2342,9.0440
lightgbm,Light Gradient Boosting Machine,165.2951,77087.8029,276.8805,0.9320,0.4512,0.3655,2.5040
dt,Decision Tree Regressor,205.3351,143839.7876,378.7812,0.8730,0.3973,0.2858,2.1590
gbr,Gradient Boosting Regressor,280.8619,168806.8917,410.3001,0.8509,0.6230,0.7152,3.3630
br,Bayesian Ridge,396.0140,328298.9878,572.5413,0.7097,0.8280,1.1051,1.9540
ridge,Ridge Regression,396.0214,328328.7099,572.5663,0.7097,0.8268,1.1048,2.5820
lr,Linear Regression,397.2889,329362.9727,573.4699,0.7088,0.8416,1.1098,5.6620
lasso,Lasso Regression,395.7796,330051.0717,574.0533,0.7082,0.8182,1.0950,3.3890


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
tuned_top5 = [tune_model(i) for i in top5]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,145.9250,54605.8554,233.6790,0.9484,0.4411,0.2981
1,154.2595,71182.1418,266.7998,0.9445,0.5141,0.3663
2,141.8805,53122.4766,230.4831,0.9524,0.5295,0.3509
3,144.2488,65929.6030,256.7676,0.9396,0.4319,0.3154
4,146.7461,59398.2063,243.7175,0.9427,0.4192,0.3173
5,151.2253,56908.5973,238.5552,0.9505,0.4839,0.3030
6,144.2459,53149.5261,230.5418,0.9525,0.4628,0.3061
7,144.6952,52637.0806,229.4277,0.9545,0.4126,0.2734
8,155.9641,59441.2615,243.8058,0.9506,0.4722,0.3204


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,473.0053,364974.9057,604.1315,0.6553,0.9957,1.7737
1,483.7055,404880.9705,636.3026,0.6845,1.0379,1.9416
2,482.3309,376522.2901,613.6141,0.6623,1.0345,1.8849
3,471.5462,369632.5390,607.9741,0.6612,1.0426,1.9382
4,472.3692,351093.4898,592.5314,0.6615,1.0613,1.9492
5,471.5250,347594.1702,589.5712,0.6979,1.0061,1.7959
6,472.3641,347483.0940,589.4770,0.6894,1.0321,1.8828
7,472.7777,368211.7237,606.8045,0.6818,0.9769,1.6968
8,489.0125,380424.1233,616.7853,0.6836,1.0036,1.8245


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,293.7876,229635.6197,479.2031,0.7831,0.5229,0.5078
1,284.6995,235433.9218,485.2153,0.8165,0.5016,0.4927
2,262.0560,198324.4332,445.3363,0.8221,0.5254,0.5106
3,245.2399,178517.0174,422.5127,0.8364,0.4625,0.4115
4,278.7824,208203.2902,456.2930,0.7992,0.5066,0.4382
5,288.0751,232587.4883,482.2733,0.7978,0.4900,0.4620
6,310.1813,248075.2715,498.0716,0.7782,0.4922,0.4776
7,266.8565,197131.2707,443.9947,0.8296,0.4899,0.4411
8,311.6224,283854.3454,532.7798,0.7639,0.5132,0.4793


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
top5

[XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=6281, ...),
 ExtraTreesRegressor(n_jobs=-1, random_state=6281),
 RandomForestRegressor(n_jobs=-1, random_state=6281),
 LGBMRegressor(random_state=6281),
 DecisionTreeRegressor(random_state=6281)]

In [ ]:
blender_top5 = blend_models(estimator_list = tuned_top5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,137.1728,54903.6937,234.3154,0.9481,0.3616,0.2339
1,145.8431,80446.2222,283.6304,0.9373,0.3629,0.2570
2,136.6486,65251.2456,255.4432,0.9415,0.3527,0.2410
3,135.5725,69043.6193,262.7615,0.9367,0.3416,0.2264
4,136.1257,60961.6922,246.9042,0.9412,0.3718,0.2250
5,137.9343,55243.3517,235.0390,0.9520,0.3216,0.2264
6,141.0283,56415.5889,237.5197,0.9496,0.3127,0.2099
7,134.3842,56036.5400,236.7204,0.9516,0.3191,0.2074
8,140.7178,58628.7766,242.1338,0.9512,0.3351,0.2308


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
final_model = finalize_model(blender_top5)
prediction = predict_model(final_model, data=test)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,185.3425,98455.9235,313.7769,0.9219,0.3923,0.3012


In [ ]:
from pycaret.utils import check_metric
check_metric(prediction['cnt'], prediction['Label'], metric = 'RMSE')

최종 파이프라인

In [ ]:
class Regressor(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(nn.Linear(62,30),
                               nn.ReLU(),
                               nn.Linear(30,30),
                               nn.ReLU(),
                               nn.Linear(30,10),
                               nn.Dropout(0.2),
                               nn.ReLU(),
                               nn.Linear(10,1),
                               )

  def forward(self,x):
    x=self.model(x)
    return x

In [ ]:
def pipeline(cols, cat_cols, method='lr'):
  cat_transformer = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

  num_cols = list(set(cols)-set(cat_cols))
  num_cols.sort()
  num_transformer = RobustScaler()

  preprocessor = ColumnTransformer(transformers=[("num",num_transformer,num_cols),
                                                 ("cat",cat_transformer,cat_cols)])


  class FloatTransformer(BaseEstimator, TransformerMixin):
      def __init__(self):
            pass
      def fit(self, X, y=None):
            return self
      def transform(self, x):
            return np.array(x, dtype=np.float32)

  if method == 'lr':
    ml = LinearRegression(fit_intercept=True)
  elif method == 'rf':
    ml = RandomForestRegressor()
  elif method == 'xgb':
    ml = XGBRegressor()
  elif method == 'etr':
    ml = ExtraTreesRegressor()
  elif method == 'torch':
    ml = NeuralNetRegressor(Regressor(),max_epochs=100,criterion=nn.MSELoss(),device='cuda:0',optimizer=optim.Adam,optimizer__lr = 0.01)

  model = Pipeline(steps=[("preprocessor",preprocessor),
                          ("float64to32",FloatTransformer()),
                          ("ml",ml)])

  return model

In [ ]:
dnn = pipeline(list(cols),list(cat_cols), method="torch")
dnn.fit(X_train, y_train.astype(np.float32).values.reshape(-1, 1))
dnn

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1  1651424.9628   591701.2553  0.4086
      2   490751.9420   230461.5590  0.4257
      3   370976.6354  1000676.5953  0.4383
      4   387732.5808   222515.3840  0.4182
      5   292143.1406   808058.3045  0.4560
      6   297768.4681   885539.1549  0.4199
      7   300457.9880   228039.7459  0.3304
      8   264283.4468   237307.7781  0.2772
      9   249583.2473   310388.4720  0.3123
     10   227059.5658   332298.7372  0.3133
     11   207487.7094   201128.3199  0.3043
     12   186450.0157    88777.0517  0.2710
     13   174225.2065    56534.7343  0.2944
     14   188114.7724   142650.6150  0.2712
     15   193821.1512   193984.3156  0.2853
     16   186997.0275   287987.6131  0.2823
     17   210022.3903   211340.6474  0.2959
     18   202175.7636    86736.8034  0.2922
     19   192088.4107   144180.0138  0.2845
     20   187779.2569   191777.7550  0.2933
     21   183682.0074   286883.2

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['hum', 't1', 't2',
                                                   'wind_speed']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['weather_code', 'is_holiday',
                                                   'is_weekend', 'season',
                                                   'month', 'hour', 'day'])])),
                ('float64to32', FloatTransformer()),
                ('ml',
                 <class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Regressor(
    (model): Sequential(
      (0): Linear(in_features=62, out_features=30, bias=True)
      (1): ReLU()
      (2): Linear(in_features=30, out_features=30, bias=True)
      (3): ReLU()
      (4): Linear(in_features=30, out_features=10, bias=True)
      (5): Dropout(p=0.2, inplace=False)
      (6): ReLU()
      (7): Linear(in_features=10, out_features=1, bias=True)
    )
  ),
))])

In [ ]:
y_pred = dnn.predict(X_test)

In [ ]:
rmse_dnn = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_dnn

335.8863693839808

In [ ]:
dnn = pipeline(list(cols),list(cat_cols), method="torch")

In [ ]:
params = {
    'ml__optimizer__lr':[0.1,0.01,0.001],
    'ml__batch_size' : [10,40,100],
    'ml__max_epochs' : [10,50,100]
}

In [ ]:
grid_dnn=GridSearchCV(estimator=dnn,param_grid=params,scoring='neg_root_mean_squared_error',n)jobs=-1).fit(X_train,y_train.astype(np.float32).values.reshape(-1, 1))

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1   713441.4889   526839.2875  17.0489
      2   431972.8272   373917.8433  17.2325
      3   445539.1116   362316.5273  17.6262
      4   436735.7435   503112.1658  17.0257
      5   485395.1292   273038.6490  17.2063
      6   466166.4968   601146.1291  17.9661
      7   566677.8877   628465.3214  17.1925
      8   564301.5693   722626.2751  16.9596
      9   596399.4235   925849.0206  17.6699
     10   566473.6917   671112.2843  17.3954
  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1   631838.1683   388996.5056  17.0201
      2   518105.5582   290737.9581  17.7506
      3   503984.0727   386250.9533  17.1121
      4   497682.2690   321461.2319  17.0161
      5   485072.5932   268007.9639  17.3094
      6   541032.9924   265625.1579  17.4326
      7   462513.3228   219738.0531  16.9709
      8   467980.4656   401229.0093  17.0383
      9   

In [ ]:
y_pred=grid_dnn.predict(X_test)
rmse_gr_dnn = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_gr_dnn

380.4623336840237

In [ ]:
print("Best: %f using %s" % (grid_dnn.best_score_, grid_dnn.best_params_))
means = grid_dnn.cv_results_['mean_test_score']
stds = grid_dnn.cv_results_['std_test_score']
params = grid_dnn.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))